In [4]:
import re
import requests
import json
import re

from bs4 import BeautifulSoup

In [5]:
class WCAG_base(object):
    _private = []

    def as_dict(self):
        safe_dict = {
            k: self.__dict__[k]
            for k
            in self.__dict__ if k not in self._private + ['_private']
        }
        return safe_dict

    def __repr__(self):
        return json.dumps(self.as_dict())
  

class WCAG(WCAG_base):
    _private = ['_soup']

    def __init__(self, url):
        r = requests.get(url)
        self._soup = BeautifulSoup(r.text, 'html.parser')        
        self.principles=[Principle(p).as_dict() for p in self._soup.body.find_all('section', class_='principle')]
        
    def to_json(self, filename):
        with open(filename, 'w') as f:
            json.dump(self.as_dict(), f, indent=2)

class WCAGElement(WCAG_base):
    _private = ['_soup']

    def __init__(self, soup, title_element):
        self._soup = soup
        self.id = self._soup['id']
        label = self._soup.find(class_='secno').text
        self.label = re.sub(r'[\.\s]+$', '', label)
        self.title = self._soup.find(
            title_element).text.replace(label, '').strip()
        self.description = re.sub(
            r'\s+', ' ', string=self._soup.p.text).strip()


class Principle(WCAGElement):
    def __init__(self, soup):
        super().__init__(soup, 'h2')
        guidelines = self._soup.find_all(
            'section', class_="guideline", recursive=False)
        self.guidelines = [Guideline(g).as_dict() for g in guidelines]


class Guideline(WCAGElement):
    def __init__(self, soup):
        super().__init__(soup, 'h3')
        criteria = self._soup.find_all(
            'section', class_="guideline", recursive=False)
        self.criteria = [Criterion(c).as_dict() for c in criteria]


class Criterion(WCAGElement):
    def __init__(self, soup):
        super().__init__(soup, 'h4')
        try:
            self.level = re.sub(pattern=r'^.*?(A+).*$',
                                repl='\g<1>',
                                string=self._soup.find(
                                    'p', class_="conformance-level").text
                                )
        except:
            print(f"{self.title} has no level")

        self.description = re.sub(r'\s+', ' ', string=self._soup.find('p', class_="").text).strip()

In [7]:
WCAG('https://www.w3.org/TR/WCAG21/').to_json('wcag21.json')
WCAG('https://www.w3.org/TR/WCAG22/').to_json('wcag22.json')



Parsing (Obsolete and removed) has no level


,principles
0,"{'id': 'perceivable', 'label': '1', 'title': '..."
1,"{'id': 'operable', 'label': '2', 'title': 'Ope..."
2,"{'id': 'understandable', 'label': '3', 'title'..."
3,"{'id': 'robust', 'label': '4', 'title': 'Robus..."
